In [ ]:
import pdfplumber
def extract_text_from_pdf(pdf_path):
    all_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                all_text += text + "\n\n\n\n"
    return all_text

In [ ]:
# Path to your PDF file
pdf_path = 'data/pdf4.pdf'

# Extract text from the PDF
extracted_text = extract_text_from_pdf(pdf_path)
print(extracted_text)

In [ ]:
# from langchain import PromptTemplate
from langchain_groq import ChatGroq
# from langchain.chains.summarize import load_summarize_chain
# from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv
load_dotenv()

## Load the Groq API
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
# summarization_model = ChatGroq(model_name = "llama-3.1-70b-versatile")
summarization_model = ChatGroq(model_name = "llama-3.1-8b-instant")


In [ ]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=20)
# chunks = text_splitter.create_documents([extracted_text])

In [ ]:
# len(chunks)

In [ ]:
from deep_translator import GoogleTranslator
from transformers import pipeline
import textwrap

# Load the summarization model
# summarization_model = pipeline("summarization", model="facebook/bart-large-cnn")

def translate_text(text):
    """Translate Chinese text to English using Deep Translator."""
    try:
        translated_text = GoogleTranslator(source='zh-CN', target='en').translate(text)
        return translated_text
    except Exception as e:
        print(f"Error during translation: {e}")
        return ""

def summarize_text(text):
    text = f"Summarize the following text in 50-60 words only. '{text}'"
    """Summarize English text."""
    try:
        # Check if text is not empty
        if len(text) > 0:
            # Ensure the text is within the summarization model's limits
            summary = summarization_model.invoke(text)
            return summary
        else:
            print("No text to summarize.")
            return ""
    except Exception as e:
        print(f"Error during summarization: {e}")
        return ""

def process_long_text(long_text, chunk_size=1000):
    """Process a long text: translate and summarize."""
    # Split the long text into manageable chunks
    chunks = textwrap.wrap(long_text, width=chunk_size, break_long_words=False)
    
    translated_chunks = []
    for chunk in chunks:
        translated_chunk = translate_text(chunk)
        if translated_chunk:  # Ensure translated chunk is not empty
            translated_chunks.append(translated_chunk)
    
    # Combine all translated chunks
    combined_translated_text = " ".join(translated_chunks)
    print(combined_translated_text)

    # Only summarize if we have valid translated text
    if combined_translated_text.strip():  # Check if there's any valid translated text
        summary = summarize_text(combined_translated_text)
        return summary
    else:
        return "No valid translated text available for summarization."

In [ ]:
long_chinese_text = extracted_text # Your long Chinese text here
print(long_chinese_text)

In [ ]:
# Process the long text
english_summary = process_long_text(long_chinese_text)

In [ ]:
from pprint import pprint
pprint(english_summary.content)

MAP REDUCE

In [ ]:
from langchain.document_loaders import PyPDFLoader
pdf_file = "data/pdf1.pdf"
pdf_loader = PyPDFLoader(pdf_file)
pages = pdf_loader.load_and_split()
pages

In [ ]:
pages[0].page_content

In [ ]:
from langchain.prompts import PromptTemplate
map_prompt_template = """
Write a summary of this chunk of text that includes the main points and any important details. just write in 40-50 words.
{text}
"""

main_prompt = PromptTemplate(template = map_prompt_template, input_variables=["text"])

combined_prompt_template = """
Write a concise summary of the following text delimited by triple backquotes.
Return your response in bullet points which covers the key points of the text.
```{text}```
BULLET POINT SUMMARY:
"""

combine_prompt = PromptTemplate(
    template=combined_prompt_template, input_variables=["text"]
)

In [ ]:
# from langchain import PromptTemplate
from langchain_groq import ChatGroq
# from langchain.chains.summarize import load_summarize_chain
# from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv
load_dotenv()

## Load the Groq API
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
summarization_model = ChatGroq(model_name = "llama-3.1-70b-versatile")
# summarization_model = ChatGroq(model_name = "llama-3.1-8b-instant")

In [ ]:
from langchain.chains import load_summarize_chain
map_reduce_cahin = load_summarize_chain(
    summarization_model,
    chain_type="map_reduce",
    combine_prompt=combine_prompt,
    return_intermediate_steps = True,    
)

In [ ]:
from deep_translator import GoogleTranslator 
from langchain_core.documents import Document
# Function to translate and reformat documents
def translate_documents(documents):
    translated_docs = []
    
    for doc in documents:
        # Translate the content
        translated_content = GoogleTranslator(source='zh-CN', target='en').translate(doc.page_content)
        
        # Create a new Document with the translated content
        translated_doc = Document(metadata=doc.metadata, page_content=translated_content)
        
        # Append the new Document to the list
        translated_docs.append(translated_doc)
    
    return translated_docs
translated_docs = translate_documents(pages)

In [ ]:
translated_docs

In [ ]:
len(translated_docs)

GENERATE PDF OF TRANSLATED TEXT

In [ ]:
from fpdf import FPDF
FONT_SIZE = 8
FONT_STYLE = "Arial"
MARGIN = 10
LINE_SPACE = 5
PDF_NAME = "multi_page_document.pdf"

# Create a PDF object
pdf = FPDF(orientation='P', unit='mm', format='A4') # P:portrait, L:Landscape, 'mm': Millimeters (default), 'pt': Points (1 point = 1/72 of an inch), 'cm': Centimeters, 'in': Inches
pdf.set_auto_page_break(auto=True, margin=MARGIN)
pdf.set_font(FONT_STYLE, size=FONT_SIZE)

# Loop through each Document object and add its content to a new page
for doc in translated_docs:
    pdf.add_page()
    pdf.multi_cell(0, LINE_SPACE, doc.page_content.encode('latin-1', 'replace').decode('latin-1'))  # Encode with 'replace' to handle unsupported chars

# Save the PDF file
pdf_filename = PDF_NAME
pdf.output(pdf_filename)

print(f"PDF '{pdf_filename}' created successfully.")

In [ ]:
# map_reduce_output = map_reduce_cahin({"input_documents":pages})
# map_reduce_output

In [ ]:
translated_docs[0:5]

In [ ]:
response = map_reduce_cahin({"input_documents":translated_docs})
response['output_text']

In [ ]:
print(response['output_text'])

In [ ]:
from pathlib import Path
final_mp_data = []
for doc, out in zip(response["input_documents"], response["intermediate_steps"]):
    output = {}
    output["file_name"] = Path(doc.metadata["source"]).stem
    output["file_type"] = Path(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["concise_summary"] = out
    final_mp_data.append(output)

In [ ]:
import pandas as pd
pdf_mp_summary = pd.DataFrame.from_dict(final_mp_data)
pdf_mp_summary = pdf_mp_summary.sort_values(by=["file_name", "page_number"])
pdf_mp_summary.reset_index(inplace=True, drop=True)
pdf_mp_summary.head()

PAGE WISE SUMMARY

In [ ]:
index = 1
print("[Context]")
print(pdf_mp_summary["chunks"].iloc[index])
print("\n\n[Simple Summary]")
print(pdf_mp_summary["concise_summary"].iloc[index])
print("\n\n[Page number]")
print(pdf_mp_summary["page_number"].iloc[index])
print("\n\n[Source: file_name]")
print(pdf_mp_summary["file_name"].iloc[index])

PPT or PPTX READER

In [3]:
from langchain_community.document_loaders import UnstructuredPowerPointLoader

def extract_text_from_ppt_pagewise(ppt_path):
    # Load the PowerPoint file in elements mode
    loader = UnstructuredPowerPointLoader(ppt_path, mode="elements")
    documents = loader.load()

    # # Dictionary to hold text grouped by page number
    # pagewise_text = {}

    # # Group text based on page_number
    # for doc in documents:
    #     page_number = doc.metadata.get("page_number", None)

    #     # Skip if page_number is None or replace with a default page number (e.g., 0)
    #     if page_number is None or not doc.page_content.strip():
    #         continue  # You can use a default value or continue to skip None values.

    #     # Initialize the key if not already present and concatenate text
    #     if page_number not in pagewise_text:
    #         pagewise_text[page_number] = doc.page_content
    #     else:
    #         pagewise_text[page_number] += f" {doc.page_content}"  # Concatenate with a space for better readability

    # # Convert the grouped text to a list, sorted by page number
    # pages_text = [text for _, text in sorted(pagewise_text.items())]

    return documents

# Example usage
# ppt_path = "data/20151014180939084157.ppt"
ppt_path = "data/20151110.pptx"
page_text_list = extract_text_from_ppt_pagewise(ppt_path)
page_text_list
# # Print the concatenated text for each page
# for i, page_text in enumerate(page_text_list, 1):
#     print(f"Page {i}: {page_text}")

[Document(metadata={'source': 'data/20151110.pptx', 'category_depth': 1, 'file_directory': 'data', 'filename': '20151110.pptx', 'last_modified': '2024-09-27T15:20:20', 'page_number': 1, 'languages': ['zho'], 'filetype': 'application/vnd.openxmlformats-officedocument.presentationml.presentation', 'category': 'Title', 'element_id': 'f827acccd3b2b1d66b3dbaeda26a70f6'}, page_content='学习资料  仅供参考'),
 Document(metadata={'source': 'data/20151110.pptx', 'category_depth': 1, 'file_directory': 'data', 'filename': '20151110.pptx', 'last_modified': '2024-09-27T15:20:20', 'page_number': 1, 'languages': ['zho'], 'filetype': 'application/vnd.openxmlformats-officedocument.presentationml.presentation', 'category': 'Title', 'element_id': '6343aa076763b9267967011c6d94cf0a'}, page_content='认真学好用好《准则》《条例》'),
 Document(metadata={'source': 'data/20151110.pptx', 'category_depth': 1, 'file_directory': 'data', 'filename': '20151110.pptx', 'last_modified': '2024-09-27T15:20:20', 'page_number': 1, 'languages': ['z

REFINE 

In [ ]:
from langchain.prompts import PromptTemplate

question_prompt_template = """
Please provide a summary of the following text.
TEXT: {text}
SUMMARY:
"""

question_prompt = PromptTemplate(template = question_prompt_template, input_variables=["text"])

refine_prompt_template = """
Write a concise summary of the following text delimited by triple backquotes.
Return your response in bullet points which covers the key points of the text.
```{text}```
BULLET POINT SUMMARY:
"""

refine_prompt = PromptTemplate(
    template=refine_prompt_template, input_variables=["text"]
)

In [ ]:
refine_chain = load_summarize_chain(
    summarization_model,
    chain_type="refine",
    question_prompt=question_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps = True,
)

In [ ]:
refine_outputs = refine_chain({"input_documents": translated_docs})

In [ ]:
print(refine_outputs['output_text'])

In [ ]:
from pathlib import Path
final_refine_data = []
for doc, out in zip(refine_outputs["input_documents"], refine_outputs["intermediate_steps"]):
    output = {}
    output["file_name"] = Path(doc.metadata["source"]).stem
    output["file_type"] = Path(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["concise_summary"] = out
    final_refine_data.append(output)

In [ ]:
import pandas as pd
pdf_mp_summary = pd.DataFrame.from_dict(final_refine_data)
pdf_mp_summary = pdf_mp_summary.sort_values(by=["file_name", "page_number"])
pdf_mp_summary.reset_index(inplace=True, drop=True)
pdf_mp_summary.head()

In [ ]:
index = 1
print("[Context]")
print(pdf_mp_summary["chunks"].iloc[index])
print("\n\n[Simple Summary]")
print(pdf_mp_summary["concise_summary"].iloc[index])
print("\n\n[Page number]")
print(pdf_mp_summary["page_number"].iloc[index])
print("\n\n[Source: file_name]")
print(pdf_mp_summary["file_name"].iloc[index])

In [6]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3:latest")

In [7]:
llm("hi")

/tmp/ipykernel_475655/1876039839.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm("hi")


"Hi! It's nice to meet you. Is there something I can help you with, or would you like to chat?"

GENERATING THE SAME WITH THE COORDINATES

Trial 1

In [35]:
import fitz  # PyMuPDF
from deep_translator import GoogleTranslator

def wrap_text(text, font_size, max_width):
    """Wrap text to fit within the specified maximum width."""
    words = text.split()
    wrapped_lines = []
    current_line = ""

    for word in words:
        # Calculate the width of the current line with the new word added
        test_line = f"{current_line} {word}".strip()  # Create a test line
        width = fitz.get_text_length(test_line, fontsize=font_size)  # Get the text width

        if width <= max_width:
            current_line = test_line  # Add the word to the current line
        else:
            # If adding the word exceeds the max width, append the current line and start a new one
            wrapped_lines.append(current_line)
            current_line = word  # Start a new line with the current word

    # Append any remaining text as the last line
    if current_line:
        wrapped_lines.append(current_line)

    return wrapped_lines

# Open the PDF document
doc = fitz.open("data/pdf2.pdf")

# Extract text and coordinates
text_elements = []
for page_num in range(doc.page_count):
    page = doc[page_num]
    blocks = page.get_text("blocks")  # Extract blocks of text with coordinates
    for block in blocks:
        x0, y0, x1, y1, text, _, _ = block  # Extract bounding box and text
        text_elements.append({"page": page_num, "x0": x0, "y0": y0, "x1": x1, "y1": y1, "text": text})

doc.close()


Trial 2

better Text formatting

In [36]:
translated_elements = []
for element in text_elements:
    translated_text = GoogleTranslator(source='auto', target='en').translate(element["text"])
    
    # Check if translated_text is None
    if translated_text is not None:
        element["translated_text"] = translated_text
        translated_elements.append(element)
    else:
        # Handle the case where translation fails
        print(f"Translation failed for text: {element['text']}")
        element["translated_text"] = ""  # Assign empty string if translation fails
        translated_elements.append(element)


Translation failed for text: （%） 

Translation failed for text: （%） 

Translation failed for text: （%） 

Translation failed for text: （%） 



In [55]:
# Create a new PDF document
new_doc = fitz.open()  # Start a new document

# Load the original document again to copy pages
original_doc = fitz.open("data/pdf2.pdf")
shift_left = 60

for element in translated_elements:
    page_num = element["page"]
    if len(new_doc) <= page_num:  # Create pages as needed
        new_page = new_doc.new_page(width=original_doc[page_num].rect.width,
                                    height=original_doc[page_num].rect.height)
    else:
        new_page = new_doc[page_num]

    # Get position and text
    x0, y0, x1, y1 = element["x0"], element["y0"], element["x1"], element["y1"]
    translated_text = element["translated_text"]

    # Define maximum width for the translated text
    max_width = x1 - x0 - shift_left

    # Only wrap text if it's not empty
    if translated_text:
        wrapped_lines = wrap_text(translated_text, font_size=6, max_width=max_width)

        # Insert each line of wrapped text at the specified position
        for line_num, line in enumerate(wrapped_lines):
            new_page.insert_text((x0 - shift_left, y0 + line_num * 10), line, fontsize=10)  # Adjust vertical spacing as necessary
    else:
        # Handle the case where there's no translated text
        new_page.insert_text((x0 - shift_left, y0), "Translation failed.", fontsize=10)
        
# Save the new PDF
new_doc.save("translated_document.pdf")
new_doc.close()
original_doc.close()


In [52]:
import fitz  # PyMuPDF
from deep_translator import GoogleTranslator

def wrap_text(text, font_size, max_width):
    """Wrap text to fit within the specified maximum width."""
    words = text.split()
    wrapped_lines = []
    current_line = ""

    for word in words:
        # Calculate the width of the current line with the new word added
        test_line = f"{current_line} {word}".strip()  # Create a test line
        width = fitz.get_text_length(test_line, fontsize=font_size)  # Get the text width

        if width <= max_width:
            current_line = test_line  # Add the word to the current line
        else:
            # If adding the word exceeds the max width, append the current line and start a new one
            wrapped_lines.append(current_line)
            current_line = word  # Start a new line with the current word

    # Append any remaining text as the last line
    if current_line:
        wrapped_lines.append(current_line)

    return wrapped_lines

# Open the PDF document
doc = fitz.open("data/pdf2.pdf")

# Extract text and coordinates
text_elements = []
for page_num in range(doc.page_count):
    page = doc[page_num]
    blocks = page.get_text("blocks")  # Extract blocks of text with coordinates
    for block in blocks:
        x0, y0, x1, y1, text, _, _ = block  # Extract bounding box and text
        text_elements.append({"page": page_num, "x0": x0, "y0": y0, "x1": x1, "y1": y1, "text": text})

doc.close()

translated_elements = []
for element in text_elements:
    translated_text = GoogleTranslator(source='auto', target='en').translate(element["text"])
    
    # Check if translated_text is None
    if translated_text is not None:
        element["translated_text"] = translated_text
        translated_elements.append(element)
    else:
        # Handle the case where translation fails
        print(f"Translation failed for text: {element['text']}")
        element["translated_text"] = ""  # Assign empty string if translation fails
        translated_elements.append(element)

# Create a new PDF document
new_doc = fitz.open()  # Start a new document

# Load the original document again to copy pages
original_doc = fitz.open("data/pdf2.pdf")
shift_left = 75

for element in translated_elements:
    page_num = element["page"]
    if len(new_doc) <= page_num:  # Create pages as needed
        new_page = new_doc.new_page(width=original_doc[page_num].rect.width,
                                    height=original_doc[page_num].rect.height)
    else:
        new_page = new_doc[page_num]

    # Get position and text
    x0, y0, x1, y1 = element["x0"], element["y0"], element["x1"], element["y1"]
    translated_text = element["translated_text"]

    # Define maximum width for the translated text
    max_width = x1 - x0 - shift_left

    # Only wrap text if it's not empty
    if translated_text:
        wrapped_lines = wrap_text(translated_text, font_size=6, max_width=max_width)

        # Insert each line of wrapped text at the specified position
        for line_num, line in enumerate(wrapped_lines):
            new_page.insert_text((x0 - shift_left, y0 + line_num * 10), line, fontsize=6)  # Adjust vertical spacing as necessary
    else:
        # Handle the case where there's no translated text
        new_page.insert_text((x0 - shift_left, y0), "Translation failed.", fontsize=10)

# Save the new PDF
new_doc.save("translated_document.pdf")
new_doc.close()
original_doc.close()


Translation failed for text: （%） 

Translation failed for text: （%） 

Translation failed for text: （%） 

Translation failed for text: （%） 



Above code is good for text only not working for the tables and graphs